In [55]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import pandas as pd


In [116]:
# specify paths and load in data
data_path = f'{os.getcwd()}/data/'
output_path = f"{data_path}/frequencies/"

dists_path = f"{data_path}/20240626_allROIs_25px_Iris_neighbours_distanceCalculation_allCells.csv"
coms_path = f"{data_path}/Rphenograph_Megan_26June_output_20clusters_k250_13ct_fractions.csv"

comms = pd.read_csv(coms_path)[['source_ID', 'cluster']]
dists = pd.read_csv(dists_path)

# Merge dataframe of neighbour pairs with dataframe of source cell communities
dists = dists[['treatment','ROI_ID','source_cluster', 'target_cluster', 'source_ID']]
dists_merged = pd.merge(dists, comms, on='source_ID', how='inner').rename(columns={'cluster':'community'})

dists_merged

,treatment,ROI_ID,source_cluster,target_cluster,source_ID,community
0,MRTX+PD1+CTLA-4,04_MRTX+PD1+CTLA-4,Endothelium,B cells,Cell33341,6
1,MRTX+PD1+CTLA-4,04_MRTX+PD1+CTLA-4,Endothelium,Endothelium,Cell33341,6
2,MRTX+PD1+CTLA-4,04_MRTX+PD1+CTLA-4,Endothelium,T cells CD8,Cell33341,6
3,MRTX+PD1+CTLA-4,04_MRTX+PD1+CTLA-4,Endothelium,Macrophages type 1,Cell33341,6
4,MRTX+PD1+CTLA-4,04_MRTX+PD1+CTLA-4,Endothelium,Endothelium,Cell33341,6
...,...,...,...,...,...,...
1466797,MRTX+PD1,01_MRTX+PD1,T cells CD8,T cells CD8,Cell11855,4
1466798,MRTX+PD1,01_MRTX+PD1,T cells CD8,Macrophages type 1,Cell11855,4
1466799,MRTX+PD1,01_MRTX+PD1,T cells CD8,T cells CD4,Cell11855,4
1466800,MRTX+PD1,01_MRTX+PD1,T cells CD8,T cells CD4,Cell11855,4


In [117]:
# set number of permutations
num_perms = 500
# Significance threshold
p_threshold = 0.01
# Seed for reproducibility
np.random.seed(42)

# calculate mean number of celltype B in surrounding of celltype A's per ROI, target cell and source cell
def aggregate_histo(data):
    counts = data.groupby(by=['treatment','ROI_ID', 'source_ID', 'source_cluster', 'target_cluster']).size().reset_index(name='mean')
    means = counts.groupby(by=['treatment','ROI_ID', 'source_cluster', 'target_cluster'])['mean'].mean().reset_index()
    return means

# Shuffle the target cells in a ROI to randomize the neighbours of the source cell
def shuffle_labels(data):
    shuffled_data = data.copy()
    new_dfs = []
    for treatment in shuffled_data['treatment'].unique():
        for roi in shuffled_data.loc[shuffled_data['treatment'] == treatment, 'ROI_ID'].unique():
            df = shuffled_data[(shuffled_data['treatment'] == treatment) & (shuffled_data['ROI_ID'] == roi)].copy()
            df['target_cluster'] = np.random.permutation(df['target_cluster'])
            new_dfs.append(df)
    new_df = pd.concat(new_dfs)
    return new_df


# Shuffle the data then obtain mean of number of neighbours per celltype
# Only use for permutation not for baseline
def shuffle_and_aggregate(data):
    data_shuffled = shuffle_labels(data)
    return aggregate_histo(data_shuffled)


def calc_p_vals(dat_baseline, dat_perm, n_perm, p_thresh=0.01):
    # Merge baseline with permutation
    dat_perm = pd.merge(dat_perm, dat_baseline[['treatment','ROI_ID', 'source_cluster', 'target_cluster', 'mean']],
                        on=['treatment','ROI_ID','source_cluster', 'target_cluster'], suffixes=('_perm', '_obs'), how='outer')

    # Replace NA values with 0
    dat_perm['mean_perm'] = dat_perm['mean_perm'].fillna(0)
    dat_perm['mean_obs'] = dat_perm['mean_obs'].fillna(0)
    
    # Only for returning intermediate dataframe
    dat_perm_res = dat_perm.groupby(by=['treatment','ROI_ID','source_cluster', 'target_cluster', 'mean_obs'])['mean_perm'].mean().reset_index()

    # Calculating p values for two tailed test
    dat_stat = dat_perm.groupby(['treatment','ROI_ID','source_cluster', 'target_cluster']).apply(
        lambda x: pd.Series({
            'p_gt': 1 if x['mean_obs'].max() == 0 else (sum(x['mean_perm'] >= x['mean_obs']) + 1) / (n_perm + 1),
            'p_lt': (n_perm - sum(x['mean_perm'] > x['mean_obs']) + 1) / (n_perm + 1)
        })).reset_index()

    # Determining direction of the lowest p_value
    dat_stat['direction'] = dat_stat['p_gt'] < dat_stat['p_lt']
    dat_stat['p'] = np.where(dat_stat['direction'], dat_stat['p_gt'], dat_stat['p_lt'])
    
    # Determining significance
    dat_stat['sig'] = dat_stat['p'] < p_thresh
    dat_stat['sigval'] = dat_stat['sig'].astype(int) * np.sign(dat_stat['direction'] - 0.5)
    
    return dat_stat, dat_perm_res, dat_perm




perms = []
for i in range(num_perms):
    perms.append(shuffle_and_aggregate(dists_merged[['treatment','ROI_ID', 'source_ID', 'source_cluster', 'target_cluster']]))

# Dataframe containing all the permutations
perm = pd.concat(perms, ignore_index=True)

# Dataframe containing the baseline values
bas = aggregate_histo(dists_merged[['treatment','ROI_ID', 'source_ID', 'source_cluster', 'target_cluster']])

# Run the permutation test
dat_perm_pvals, dat_perm_res_meaned, dat_perm_original_counts = calc_p_vals(bas, perm, num_perms, p_thresh=p_threshold)



  treatment       ROI_ID source_cluster target_cluster  mean_perm  mean_obs
0  MRTX+PD1  01_MRTX+PD1        B cells        B cells   1.925373  5.649867
1  MRTX+PD1  01_MRTX+PD1        B cells        B cells   1.787966  5.649867
2  MRTX+PD1  01_MRTX+PD1        B cells        B cells   1.940625  5.649867
3  MRTX+PD1  01_MRTX+PD1        B cells        B cells   1.926254  5.649867
4  MRTX+PD1  01_MRTX+PD1        B cells        B cells   1.962209  5.649867


C:\Users\jhroe\AppData\Local\Temp\ipykernel_18048\4161448172.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dat_stat = dat_perm.groupby(['treatment','ROI_ID','source_cluster', 'target_cluster']).apply(


In [119]:
# Save the dataframes for Log FC calculation
dat_perm_pvals.to_csv("permutation_results_500_pvals_260624.csv")
dat_perm_res_meaned.to_csv("permutation_results_500_meaned_260624.csv")
dat_perm_original_counts.to_csv("permutation__500_original_values_260624.csv")